In [23]:
import pandas as pd
import folium
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"
metro_data = pd.read_csv("Delhi Metro Network new.csv")
print(metro_data.head())

   Station ID         Station Name  Distance from Start (km)          Line  \
0           1             Jhil Mil                      10.3      Red line   
1           2  Welcome [Conn: Red]                      46.8     Pink line   
2           3          DLF Phase 3                      10.0   Rapid Metro   
3           4           Okhla NSIC                      26.3  Magenta line   
4           5           Dwarka Mor                      10.2     Blue line   

  Opening Date Station Layout   Latitude  Longitude  
0     6-Apr-08       Elevated  28.675790  77.312390  
1    31-Oct-18       Elevated  28.671800  77.277560  
2    14-Nov-13       Elevated  28.493600  77.093500  
3    25-Dec-17       Elevated  28.554483  77.264849  
4    30-Dec-05       Elevated  28.619320  77.033260  


In [24]:
line_colors = {
    'Red line': 'red',
    'Blue line': 'blue',
    'Yellow line': 'beige',
    'Green line': 'green',
    'Voilet line': 'purple',
    'Pink line': 'pink',
    'Magenta line': 'darkred',
    'Orange line': 'orange',
    'Rapid Metro': 'cadetblue',
    'Aqua line': 'black',
    'Green line branch': 'lightgreen',
    'Blue line branch': 'lightblue',
    'Gray line': 'lightgray'
}
delhi_map_with_line_tooltip = folium.Map(location=[28.7041, 77.1025], zoom_start=11)

for _, row in metro_data.iterrows():
    line = row['Line']
    color = line_colors.get(line, 'black')
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station Name'],
        tooltip=f"{row['Station Name']}, {line}",
        icon=folium.Icon(color=color)
    ).add_to(delhi_map_with_line_tooltip)

# Save and open
delhi_map_with_line_tooltip.save("delhi_metro_map.html")

In [31]:
metro_data['Opening Date'] = pd.to_datetime(
    metro_data['Opening Date'],
    dayfirst=True,
    errors='coerce'
)

# Drop invalid dates (if any)
metro_data = metro_data.dropna(subset=['Opening Date'])


metro_data['Opening Year'] = metro_data['Opening Date'].dt.year
stations_per_year = metro_data['Opening Year'].value_counts().sort_index()
stations_per_year_df = stations_per_year.reset_index()
stations_per_year_df.columns = ['Year', 'Number of Stations']
fig = px.bar(
    stations_per_year_df,
    x='Year',
    y='Number of Stations',
    title="Number of Metro Stations Opened Each Year in Delhi",
    labels={'Year': 'Year', 'Number of Stations': 'Stations Opened'}
)
fig.show()


stations_per_year_df["Cumulative Stations"] = stations_per_year_df["Number of Stations"].cumsum()
fig = px.line(
    stations_per_year_df,
    x="Year",
    y="Cumulative Stations",
    title="Cumulative Growth of Delhi Metro Stations Over Time",
    labels={"Year": "Year", "Cumulative Stations": "Total Stations"},
    markers=True
)
fig.show()

In [32]:
stations_per_line = metro_data['Line'].value_counts()
total_distance_per_line = metro_data.groupby('Line')['Distance from Start (km)'].max()
avg_distance_per_line = total_distance_per_line / (stations_per_line - 1)

line_analysis = pd.concat([stations_per_line, total_distance_per_line, avg_distance_per_line], axis=1)
line_analysis.columns = ['Number of Stations', 'Total Distance (km)', 'Average Distance Between Stations (km)']

line_analysis = line_analysis.sort_values(by='Number of Stations', ascending=False).reset_index()
line_analysis.rename(columns={'index': 'Line'}, inplace=True)
print(line_analysis)

                 Line  Number of Stations  Total Distance (km)  \
0           Blue line                  49                 52.7   
1           Pink line                  38                 52.6   
2         Yellow line                  37                 45.7   
3         Voilet line                  34                 43.5   
4            Red line                  29                 32.7   
5        Magenta line                  26                 35.6   
6          Green line                  22                 24.8   
7           Aqua line                  21                 27.1   
8         Rapid Metro                  11                 10.0   
9    Blue line branch                   8                  8.1   
10        Orange line                   7                 22.7   
11          Grey line                   4                  5.2   
12  Green line branch                   3                  2.1   

    Average Distance Between Stations (km)  
0                             

In [33]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Number of Stations Per Metro Line',
'Average Distance Between Stations Per Metro Line'),
horizontal_spacing=0.2)


fig.add_trace(
    go.Bar(y=line_analysis['Line'], x=line_analysis['Number of Stations'],
           orientation='h', name='Number of Stations', marker_color='crimson'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(y=line_analysis['Line'], x=line_analysis['Average Distance Between Stations (km)'],
           orientation='h', name='Average Distance (km)', marker_color='navy'),
    row=1, col=2
)
fig.update_xaxes(title_text="Number of Stations", row=1, col=1)
fig.update_xaxes(title_text="Average Distance Between Stations (km)", row=1, col=2)
fig.update_yaxes(title_text="Metro Line", row=1, col=1)
fig.update_yaxes(title_text="", row=1, col=2)
fig.update_layout(height=600, width=1200, title_text="Metro Line Analysis", template="plotly_white")
fig.show()

In [34]:
layout_counts = metro_data['Station Layout'].value_counts()

fig = px.bar(x=layout_counts.index, y=layout_counts.values,
             labels={'x': 'Station Layout', 'y': 'Number of Stations'},
             title='Distribution of Delhi Metro Station Layouts',
             color=layout_counts.index,
             color_continuous_scale='pastel')

fig.update_layout(xaxis_title="Station Layout",
                  yaxis_title="Number of Stations",
                  coloraxis_showscale=False,
                  template="plotly_white")

fig.show()

In [35]:
metro_data["Interchange"] = metro_data["Station Name"].apply(lambda x: "Yes" if "Conn:" in x else "No")
metro_data["Clean Station Name"] = metro_data["Station Name"].str.replace(r"\[.*\]", "", regex=True).str.strip()
metro_data["Connected Lines"] = metro_data["Station Name"].str.extract(r"\[Conn:\s*(.*)\]", expand=False).fillna("None")
interchanges = metro_data[metro_data["Interchange"] == "Yes"]

interchange_summary = interchanges.groupby("Line").agg({
    "Clean Station Name": lambda x: ", ".join(x),
    "Connected Lines": lambda x: ", ".join(x),
    "Station ID": "count"
}).reset_index()


interchange_summary.columns = ["Line", "Stations", "Connected Lines", "Number of Interchanges"]
fig = px.bar(
    interchange_summary,
    x="Line",
    y="Number of Interchanges",
    title="Interchange Stations per Line (with Details)",
    color_discrete_sequence=["steelblue"],
    hover_data={
        "Stations": True,          
        "Connected Lines": True,   
        "Number of Interchanges": True,
        "Line": False
    }
)
fig.show()

In [36]:
line_colors = {
    'Red line':              '#EE1C25',   # red
    'Blue line':             '#0072BC',   # blue
    'Yellow line':           '#FFD700',   # yellow/gold
    'Green line':            '#009933',   # green
    'Voilet line':           '#92278F',   # violet (keeps your spelling)
    'Pink line':             '#E61C8C',   # pink
    'Magenta line':          '#FF00FF',   # magenta
    'Orange line':           '#FF7F00',   # orange
    'Rapid Metro':           '#5DADE2',   # light / cadet blue
    'Aqua line':             '#00FFFF',   # aqua
    'Green line branch':     '#90EE90',   # light green
    'Blue line branch':      '#ADD8E6',   # light blue
    'Gray line':             '#A7A9AC'    # gray
}


fig = px.scatter(
    interchanges,
    x="Line",
    y="Clean Station Name",
    color="Line",
    text="Clean Station Name",
    hover_data=["Connected Lines"],
    title="Interchange Stations and Connected Lines",
    color_discrete_map=line_colors
)
fig.update_traces(textposition="top center")
fig.update_layout(height=1200)
fig.show()